<a href="https://colab.research.google.com/github/RutujKhare1/CS6910_Assignment3/blob/main/FDL_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [4]:
df = pd.read_csv('/content/drive/MyDrive/FDL_A3/hin/hin_train.csv', names=['eng','hin'])

In [5]:
unique_eng_letters = set(''.join(df['eng']))
unique_hin_letters = set(''.join(df['hin']))
int_to_eng = dict(enumerate(unique_eng_letters))
eng_to_int = {char: ind+1 for ind, char in int_to_eng.items()}

int_to_hin = dict(enumerate(unique_hin_letters))
hin_to_int = {char: ind+1 for ind, char in int_to_hin.items()}

In [6]:
eng_words = df['eng']
maxlen = len(max(df['eng'], key=len))
index_eng_words = []
for eng_word in eng_words:
  index_eng_word = [eng_to_int[i] for i in eng_word]
  l = len(index_eng_word)
  index_eng_word.extend([0]*(maxlen-l))
  index_eng_words.append(index_eng_word)

In [7]:
hin_words = df['hin']
maxlen = len(max(df['hin'], key=len))
index_hin_words = []
for hin_word in hin_words:
  index_hin_word = [hin_to_int[i] for i in hin_word]
  l = len(index_hin_word)
  index_hin_word.extend([0]*(maxlen-l))
  index_hin_words.append(index_hin_word)

In [8]:
tensor_eng = torch.tensor(index_eng_words)
tensor_hin = torch.tensor(index_hin_words)

In [9]:
tensor_eng

tensor([[15, 10,  5,  ...,  0,  0,  0],
        [ 6, 14, 25,  ...,  0,  0,  0],
        [24, 14, 23,  ...,  0,  0,  0],
        ...,
        [ 5, 15,  5,  ...,  0,  0,  0],
        [15, 12,  2,  ...,  0,  0,  0],
        [ 5, 25,  8,  ...,  0,  0,  0]])

In [14]:
class GRU_Encoder(nn.Module):
  def __init__(self, input_size, hid_size, num_of_enc_layers, emb_size, batch_size, bi_direct):
    self.input_size = input_size
    self.hid_size = hid_size
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.embedding = nn.Embedding(input_size, emb_size)
    self.gru = nn.GRU(emb_size, hid_size, num_of_enc_layers, bidirectional = bi_direct)

  def forwardPropagation(self, input_data, hidden):
    embed = self.embedding(input_data).view(-1, self.batch_size, self.hid_size)
    output, hidden = self.gru(embed, hidden)
    if(self.bi_direct):
      hidden = hidden.resize(2, self.num_of_enc_layers, self.batch_size, self.hid_size)
      hidden = torch.add(hidden[0], hidden[1])/2
    return output, hidden
  
  def initialiseHidden(self):
    if(self.bi_direct):
      return torch.zeros(2*self.num_of_enc_layers, self.batch_size, self.hid_size)
    else:
      return torch.zeros(self.num_of_enc_layers, self.batch_size, self.hid_size)
  

In [11]:
class GRU_Decoder(nn.Module):
  def __init__(self, op_size, num_of_dec_layers, hid_size, batch_size, emb_size):
    self.op_size = op_size
    self.hid_size = hid_size
    self.num_of_dec_layers = num_of_dec_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.embedding = nn.Embedding(op_size, emb_size)
    self.op = nn.Linear(hid_size, op_size)
    self.softmax = nn.LogSoftmax(dim = 1)

  def forwardPropagation(self, input_data, hidden):
    embed = self.embedding(input_data).view(-1, self.batch_size, self.emb_size)
    out, hidden = self.gru(embed, hidden)
    out = self.softmax(self.op(out))
    return out, hidden

In [ ]:
def train(input_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, hid_size, cell_type):
  loss = 0
  teacher_forcing = 0.5
  for x, y in input_data:
    temp = 0
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    # x = x.T
    # y = y.T
    t_step = len(x)
    if(cell_type == 'GRU'):
      enc_hidden = encoder.initialiseHidden()
      enc_output, enc_hidden = encoder(x, enc_hidden)
      
      if(num_of_dec_layers > num_of_enc_layers):
        num = num_of_dec_layers
        dec_hidden = enc_hidden
        while(num != num_of_enc_layers):
          dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
          num -= 1
      elif(num_of_dec_layers < num_of_enc_layers):
        dec_hidden = enc_hidden[-num_of_dec_layers:]
      else:
        dec_hidden = enc_hidden

      dec_input = y[0]
      





In [12]:
def training(input_data, input_size, target_size, max_input_size, epochs, batch_size, emb_size, num_of_enc_layers, num_of_dec_layers, hid_size, cell_type, bi_direct, dropout, beam_size):
  learning_rate = 0.01
  if(cell_type == "GRU"):
    encoder = GRU_Encoder(input_size, hid_size, num_of_enc_layers, emb_size, batch_size, bi_direct)
    decoder = GRU_Decoder(target_size, num_of_dec_layers, hid_size, batch_size, emb_size)
  
  enc_optimizer = torch.optim.Adam(encoder.parameters(), learning_rate)
  dec_optimizer = torch.optim.Adam(decoder.parameters(), learning_rate)
  loss_fn = nn.CrossEntropyLoss(reduction = 'sum')
  encoder.train()
  decoder.train()
  loss_list = []
  for i in range(epochs):
    loss, encoder, decoder = train(input_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, cell_type)
    loss_list.append(loss/51200)
    print(loss)

  #train_prediction by eval






# **Experimental Area**

In [ ]:
maxlen = len(max(df['eng'], key=len))

In [ ]:
maxlen

24

In [ ]:
for i in df['eng']:
  if(len(i) > 24):
    print(i)

In [ ]:
a = [1,2,3,4]
a.append(0*4)
a

[1, 2, 3, 4, 0]